In [116]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy
import math
from string import ascii_uppercase
from string import ascii_lowercase
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models
import copy

In [117]:
corpus = pd.read_csv('corpus')

In [118]:
corp = corpus

In [119]:
def tokenize(text):
#         """Use the ToktokTokenizer to obtain tokens from the text"""
    tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

In [120]:
def get_tokens(corpus):
#         """Obtain Tokens for each document in the corpus"""
#     tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
    tokenized_doc = []
    for doc in corpus:
        tokens = tokenize(doc)
        filtered_tokens = [w for w in tokens if len(w) > 2]
        tokenized_doc.append(filtered_tokens)
    return tokenized_doc

In [121]:
corpus['tokens'] = get_tokens(corpus['clean_text'])

In [122]:
doc_tokens = corpus['tokens']

In [123]:
dictionary = gensim.corpora.Dictionary(doc_tokens)

In [124]:
corpus = [dictionary.doc2bow(text) for text in doc_tokens]

In [125]:
filtered_dict = copy.deepcopy(dictionary)

In [126]:
filtered_dict.filter_extremes(no_below=30, no_above=0.5)

In [127]:
if len(filtered_dict) > 0:
    dictionary = filtered_dict
    corpus = [dictionary.doc2bow(text) for text in doc_tokens]

In [128]:
# tfidf = gensim.models.TfidfModel(corpus, dictionary=dictionary)
# corpus = tfidf[corpus]

In [129]:
cv_scores = []
umass_scores = []
model_list = {}
max_topic_clusters = 10
metrics = pd.DataFrame()
best_model = None
best_model_clusters = None

In [130]:
for topics in range(10, max_topic_clusters + 1):
    print("Running LDA with {0} topic(s)".format(topics))
    lm = gensim.models.LdaMulticore(corpus= corpus, num_topics=topics, id2word=dictionary, passes=100,
                                    chunksize=2000, alpha='symmetric', eval_every=10,
                                    random_state=42)
    model_list[topics] = lm
    
    cm_1 = gensim.models.CoherenceModel(model=lm, texts=doc_tokens, dictionary=dictionary,coherence='c_v').get_coherence()
    cm_2 = gensim.models.CoherenceModel(model=lm, corpus=corpus, dictionary=dictionary,coherence='u_mass').get_coherence()
    lp = lm.log_perplexity(corpus)
    cv_scores.append(cm_1)
    umass_scores.append(cm_2)

Running LDA with 10 topic(s)


In [131]:
metrics = pd.DataFrame(
            {'topics': range(10, max_topic_clusters + 1),'cv_score': cv_scores,
             'umass_score': umass_scores,'log_perplexity': lp})

In [132]:
metric='cv_score'
n_topics = metrics[metrics[metric] == metrics[metric].max()]['topics'].values[0]

In [133]:
n_topics

10

In [134]:
best_model = model_list.get(n_topics)
best_model_clusters = n_topics

In [135]:
a = best_model[corpus]

In [146]:
a[0]

[(3, 0.10826889), (5, 0.88926166)]

In [145]:
len(a)

67

In [147]:
r = [0]*10

In [148]:
r

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [149]:
l=[]
for i in range(len(a)):
    r= [0] * n_topics
    for j in a[i]:
        r[j[0]] =j[1]
    l.append(r)
    

In [153]:
df = pd.DataFrame(l)

In [151]:
df = pd.concat([corp['resource_id'], df], axis=1)

In [154]:
df.style.applymap(lambda x: 'background-color : red' if x>0.5 else '')

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.000000,0,0.108265,0.000000,0.889265,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.057522,0,0.186601,0.135065,0.580097,0.000000,0.000000,0.035762,0.000000
2,0.000000,0.359127,0,0.187835,0.194695,0.156900,0.000000,0.100371,0.000000,0.000000
3,0.000000,0.108221,0,0.120200,0.041889,0.000000,0.467649,0.250675,0.000000,0.000000
4,0.872630,0.000000,0,0.000000,0.000000,0.000000,0.038427,0.086495,0.000000,0.000000
5,0.000000,0.438376,0,0.225102,0.000000,0.095605,0.049322,0.000000,0.000000,0.189981
6,0.512055,0.014576,0,0.000000,0.000000,0.000000,0.266460,0.000000,0.198269,0.000000
7,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.944079,0.000000,0.000000,0.051500
8,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.953016,0.000000,0.044973,0.000000
9,0.000000,0.000000,0,0.000000,0.000000,0.032631,0.000000,0.864744,0.095460,0.000000


In [84]:
df = pd.concat([corp['resource_id'], df], axis=1)

TypeError: cannot concatenate object of type '<class 'pandas.io.formats.style.Styler'>'; only Series and DataFrame objs are valid

In [82]:
df

,0,1,2,3,4,5,6,7,8,9
0,0.029815,0.029815,0.029815,0.029815,0.731649,0.029815,0.029815,0.029815,0.029833,0.029815
1,0.024793,0.024793,0.024793,0.024793,0.769109,0.024793,0.024793,0.024793,0.032548,0.024793
2,0.016880,0.016880,0.016880,0.016880,0.847842,0.016880,0.016880,0.016880,0.017115,0.016880
3,0.019573,0.019573,0.019573,0.019573,0.510326,0.019573,0.019573,0.019573,0.333094,0.019573
4,0.025010,0.025010,0.025010,0.025010,0.345164,0.025010,0.025010,0.025010,0.454753,0.025010
5,0.018455,0.018455,0.018455,0.018455,0.823229,0.018455,0.018455,0.018455,0.029130,0.018455
6,0.029778,0.029778,0.029778,0.029778,0.088088,0.029778,0.029778,0.029778,0.673689,0.029778
7,0.016831,0.016831,0.016831,0.016831,0.195215,0.016831,0.016831,0.016831,0.670135,0.016831
8,0.024162,0.024162,0.024162,0.024162,0.182457,0.024162,0.024162,0.024162,0.624250,0.024162
9,0.019266,0.019266,0.019266,0.019266,0.703367,0.019266,0.019266,0.019266,0.142508,0.019266


In [81]:
df = df.iloc[0:67, 1:11].style.applymap(lambda x: 'background-color : red' if x>0.5 else '')

In [49]:
lda_model = best_model

In [50]:
b = []
for i in range(10):
    l = lda_model.print_topic(i).split('"')
    d = []
    for i in range(len(l)):
        if i%2==1:
            d.append(l[i])
    b.append(d)
    print(len(d))

10
10
10
10
10
10
10
10
10
10


In [51]:
df = pd.DataFrame(b)

In [77]:
df.iloc[0:67, 1:11]

,0,1,2,3,4,5,6,7,8,9
0,0.029815,0.029815,0.029815,0.029815,0.731649,0.029815,0.029815,0.029815,0.029833,0.029815
1,0.024793,0.024793,0.024793,0.024793,0.769109,0.024793,0.024793,0.024793,0.032548,0.024793
2,0.016880,0.016880,0.016880,0.016880,0.847842,0.016880,0.016880,0.016880,0.017115,0.016880
3,0.019573,0.019573,0.019573,0.019573,0.510326,0.019573,0.019573,0.019573,0.333094,0.019573
4,0.025010,0.025010,0.025010,0.025010,0.345164,0.025010,0.025010,0.025010,0.454753,0.025010
...,...,...,...,...,...,...,...,...,...,...
62,0.021274,0.021274,0.021274,0.021274,0.714424,0.021274,0.021274,0.021274,0.115382,0.021274
63,0.018729,0.018729,0.018729,0.018729,0.831434,0.018729,0.018729,0.018729,0.018735,0.018729
64,0.019193,0.019193,0.019193,0.019193,0.827225,0.019193,0.019193,0.019193,0.019234,0.019193
65,0.026277,0.026277,0.026277,0.026277,0.763504,0.026277,0.026277,0.026277,0.026281,0.026277
